In [19]:
%load_ext rpy2.ipython


The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [4]:
# Step 1 - Import Salary Data

import pandas as pd

# Try to load the CSV file with error handling
try:
    salary_df = pd.read_csv("Total.csv")
    print("✅ Salary data loaded successfully!")
    display(salary_df.head())  # show first 5 rows for confirmation
except FileNotFoundError:
    print("❌ Error: The file 'Total.csv' was not found in the current directory.")
except pd.errors.EmptyDataError:
    print("❌ Error: The file 'Total.csv' is empty.")
except Exception as e:
    print(f"❌ An unexpected error occurred: {e}")


✅ Salary data loaded successfully!


C:\Users\HP\AppData\Local\Temp\ipykernel_4256\1367190881.py:7: DtypeWarning: Columns (2,3,4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  salary_df = pd.read_csv("Total.csv")


EmployeeName                                        JobTitle  \
0     NATHANIEL FORD  GENERAL MANAGER-METROPOLITAN TRANSIT AUTHORITY   
1       GARY JIMENEZ                 CAPTAIN III (POLICE DEPARTMENT)   
2     ALBERT PARDINI                 CAPTAIN III (POLICE DEPARTMENT)   
3  CHRISTOPHER CHONG            WIRE ROPE CABLE MAINTENANCE MECHANIC   
4    PATRICK GARDNER    DEPUTY CHIEF OF DEPARTMENT,(FIRE DEPARTMENT)   

     BasePay OvertimePay   OtherPay      Benefits   TotalPay  \
0  167411.18         0.0  400184.25  Not Provided  567595.43   
1  155966.02   245131.88  137811.38  Not Provided  538909.28   
2  212739.13   106088.18    16452.6  Not Provided  335279.91   
3    77916.0    56120.71   198306.9  Not Provided  332343.61   
4   134401.6      9737.0  182234.59  Not Provided  326373.19   

   TotalPayBenefits  Year  
0         567595.43  2011  
1         538909.28  2011  
2         335279.91  2011  
3         332343.61  2011  
4         326373.19  2011

In [3]:
!pip install pandas

In [5]:
# Step 2 - Create Employee Lookup Function

def get_employee_details(name):
    """
    Given an employee's name, return their details from the salary DataFrame.
    """
    try:
        # Search for employee (case-insensitive)
        result = salary_df[salary_df['EmployeeName'].str.lower() == name.lower()]
        
        if result.empty:
            return f"❌ No employee found with name: {name}"
        else:
            return result
    except Exception as e:
        return f"❌ An error occurred: {e}"

# Example usage:
get_employee_details("NATHANIEL FORD")


EmployeeName                                        JobTitle    BasePay  \
0  NATHANIEL FORD  GENERAL MANAGER-METROPOLITAN TRANSIT AUTHORITY  167411.18   

  OvertimePay   OtherPay      Benefits   TotalPay  TotalPayBenefits  Year  
0         0.0  400184.25  Not Provided  567595.43         567595.43  2011

In [6]:
get_employee_details("GARY JIMENEZ")


EmployeeName                         JobTitle    BasePay OvertimePay  \
1  GARY JIMENEZ  CAPTAIN III (POLICE DEPARTMENT)  155966.02   245131.88   

    OtherPay      Benefits   TotalPay  TotalPayBenefits  Year  
1  137811.38  Not Provided  538909.28         538909.28  2011

In [7]:
# Step 3 - Create a dictionary for quick lookups
employee_dict = salary_df.set_index("EmployeeName").T.to_dict()

# Updated function to use dictionary
def get_employee_details(name):
    """
    Given an employee's name, return their details using the dictionary lookup.
    """
    try:
        # Look up the employee by exact name
        details = employee_dict.get(name)
        
        if details is None:
            return f"❌ No employee found with name: {name}"
        else:
            return details
    except Exception as e:
        return f"❌ An error occurred: {e}"

# Example usage:
get_employee_details("NATHANIEL FORD")


C:\Users\HP\AppData\Local\Temp\ipykernel_4256\3243761284.py:2: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  employee_dict = salary_df.set_index("EmployeeName").T.to_dict()


{'JobTitle': 'GENERAL MANAGER-METROPOLITAN TRANSIT AUTHORITY',
 'BasePay': 167411.18,
 'OvertimePay': 0.0,
 'OtherPay': 400184.25,
 'Benefits': 'Not Provided',
 'TotalPay': 567595.43,
 'TotalPayBenefits': 567595.43,
 'Year': 2011}

In [8]:
# Step 4 - Employee lookup with full error handling

def get_employee_details(name):
    """
    Given an employee's name, return their details using the dictionary lookup.
    Includes error handling for invalid input and missing data.
    """
    try:
        # 1. Check if name is provided and is a string
        if not isinstance(name, str) or not name.strip():
            return "❌ Error: Please provide a valid employee name as a non-empty string."
        
        # 2. Look up the employee by exact name (case-sensitive match in dict keys)
        details = employee_dict.get(name)
        
        if details is None:
            return f"❌ No employee found with name: {name}"
        
        # 3. Return details as a nice dictionary
        return details
    
    except Exception as e:
        return f"❌ An unexpected error occurred: {e}"

# Example usage:
print(get_employee_details("NATHANIEL FORD"))
print(get_employee_details(""))  # test empty input
print(get_employee_details("UNKNOWN NAME"))  # test not found


{'JobTitle': 'GENERAL MANAGER-METROPOLITAN TRANSIT AUTHORITY', 'BasePay': 167411.18, 'OvertimePay': 0.0, 'OtherPay': 400184.25, 'Benefits': 'Not Provided', 'TotalPay': 567595.43, 'TotalPayBenefits': 567595.43, 'Year': 2011}
❌ Error: Please provide a valid employee name as a non-empty string.
❌ No employee found with name: UNKNOWN NAME


In [9]:
get_employee_details("GARY JIMENEZ")

{'JobTitle': 'CAPTAIN III (POLICE DEPARTMENT)',
 'BasePay': 155966.02,
 'OvertimePay': 245131.88,
 'OtherPay': 137811.38,
 'Benefits': 'Not Provided',
 'TotalPay': 538909.28,
 'TotalPayBenefits': 538909.28,
 'Year': 2011}

In [10]:
get_employee_details("DISERE PAUL")


'❌ No employee found with name: DISERE PAUL'

In [11]:
# Step 4 - Enhanced error handling for employee lookup

def get_employee_details_safe(name):
    """
    Safely returns employee details with error handling.
    """
    try:
        # Check if input is a string
        if not isinstance(name, str):
            raise TypeError("Employee name must be a string.")
        
        # Remove extra spaces and check if empty
        name = name.strip()
        if not name:
            raise ValueError("Employee name cannot be empty.")
        
        # Lookup in dictionary
        details = employee_dict.get(name)
        
        if details is None:
            return f"❌ No employee found with name: {name}"
        
        return details
    
    except TypeError as te:
        return f"❌ Type Error: {te}"
    except ValueError as ve:
        return f"❌ Value Error: {ve}"
    except Exception as e:
        return f"❌ Unexpected Error: {e}"

# Example usage:
print(get_employee_details_safe("GARY JIMENEZ"))  # valid name
print(get_employee_details_safe(""))              # empty input
print(get_employee_details_safe(12345))           # wrong data type


{'JobTitle': 'CAPTAIN III (POLICE DEPARTMENT)', 'BasePay': 155966.02, 'OvertimePay': 245131.88, 'OtherPay': 137811.38, 'Benefits': 'Not Provided', 'TotalPay': 538909.28, 'TotalPayBenefits': 538909.28, 'Year': 2011}
❌ Value Error: Employee name cannot be empty.
❌ Type Error: Employee name must be a string.


In [12]:
details = get_employee_details_safe("GARY JIMENEZ")
details


{'JobTitle': 'CAPTAIN III (POLICE DEPARTMENT)',
 'BasePay': 155966.02,
 'OvertimePay': 245131.88,
 'OtherPay': 137811.38,
 'Benefits': 'Not Provided',
 'TotalPay': 538909.28,
 'TotalPayBenefits': 538909.28,
 'Year': 2011}

In [13]:
import os
import csv

# Make sure folder exists
os.makedirs("Employee Profile", exist_ok=True)

# File path
csv_path = os.path.join("Employee Profile", "GARY_JIMENEZ.csv")

# Write to CSV
with open(csv_path, mode="w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerow(["Field", "Value"])  # header
    for key, value in details.items():
        writer.writerow([key, value])

print(f"✅ CSV saved at: {csv_path}")


✅ CSV saved at: Employee Profile\GARY_JIMENEZ.csv


In [14]:
import zipfile

zip_filename = "Employee_Profile.zip"

with zipfile.ZipFile(zip_filename, "w") as zipf:
    for root, dirs, files in os.walk("Employee Profile"):
        for file in files:
            zipf.write(os.path.join(root, file))

print(f"✅ Folder zipped into: {zip_filename}")


✅ Folder zipped into: Employee_Profile.zip


In [15]:
import os
import csv
import zipfile

def export_employee_profile(name):
    """
    Looks up an employee by name, saves their details to a CSV
    inside 'Employee Profile' folder, and zips the folder.
    """
    # Step 1 - Get details
    details = get_employee_details_safe(name)
    
    if isinstance(details, str):
        # If lookup returned an error message
        return details
    
    # Step 2 - Ensure folder exists
    folder_name = "Employee Profile"
    os.makedirs(folder_name, exist_ok=True)
    
    # Step 3 - Save details to CSV
    csv_filename = os.path.join(folder_name, f"{name.replace(' ', '_')}.csv")
    with open(csv_filename, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(["Field", "Value"])  # header
        for key, value in details.items():
            writer.writerow([key, value])
    
    # Step 4 - Zip the folder
    zip_filename = "Employee_Profile.zip"
    with zipfile.ZipFile(zip_filename, 'w') as zipf:
        for root, dirs, files in os.walk(folder_name):
            for file in files:
                zipf.write(os.path.join(root, file))
    
    return f"✅ Profile for '{name}' exported and zipped successfully."

# Example usage:
print(export_employee_profile("GARY JIMENEZ"))


✅ Profile for 'GARY JIMENEZ' exported and zipped successfully.


In [16]:
export_employee_profile("EMPLOYEE NAME HERE")


'❌ No employee found with name: EMPLOYEE NAME HERE'

In [17]:
export_employee_profile("GARY JIMENEZ")


"✅ Profile for 'GARY JIMENEZ' exported and zipped successfully."

In [18]:
%load_ext rpy2.ipython


Error importing in API mode: ImportError('On Windows, cffi mode "ANY" is only "ABI".')
Trying to import in ABI mode.


In [20]:
%%R
# Step 6 - Unzip and display CSV in R

# Path to the zip file
zip_file <- "Employee_Profile.zip"

# Unzip into a folder
unzip(zip_file, exdir = "Unzipped_Employee_Profile")

# List files to confirm
files <- list.files("Unzipped_Employee_Profile/Employee Profile", full.names = TRUE)
print(files)

# Read the first CSV found
employee_data <- read.csv(files[1])
print(employee_data)


[1] "Unzipped_Employee_Profile/Employee Profile/GARY_JIMENEZ.csv"
             Field                           Value
1         JobTitle CAPTAIN III (POLICE DEPARTMENT)
2          BasePay                       155966.02
3      OvertimePay                       245131.88
4         OtherPay                       137811.38
5         Benefits                    Not Provided
6         TotalPay                       538909.28
7 TotalPayBenefits                       538909.28
8             Year                            2011
